In [0]:
import re
import ast
import math
import statistics
train_source_indices_directory= 'Data/KVR/train_indices_predictions.txt'
dev_source_indices_directory='Data/KVR/dev_indices_predictions.txt'
test_source_indices_directory='Data/KVR/test_indices_predictions.txt'

train_target_indices_directory= 'Data/KVR/train_indices_final.txt'
dev_target_indices_directory='Data/KVR/dev_indices_final.txt'
test_target_indices_directory='Data/KVR/test_indices_final.txt'

train_source_file_directory= 'Data/KVR/train_regex_kb_indices_edited.txt'
dev_source_file_directory= 'Data/KVR/dev_regex_kb_indices.txt'
test_source_file_directory= 'Data/KVR/test_regex_kb_indices.txt'

def count_dialogue_per_convo(source_file):
    with open(source_file,'r') as reader:
        source_text=reader.read()

        num_dialogues_convo=[]

        split_dialogue_regex= re.compile('\n\n')
        split_line_regex= re.compile('\n')
        match_dialogue_regex= re.compile(r'^[1-9]\d*$')

        dialogues= split_dialogue_regex.split(source_text)

        for dialogue in dialogues:
            if dialogue:
                num_dialogues =0
                lines = split_line_regex.split(dialogue)
                for line in lines:
                    line=line.strip()
                    if line:
                        
                        if '#' in line:
                            continue

                        line_indicator, _sentence = line.split(' ',1)
                        if line_indicator == '0':
                            continue

                        elif match_dialogue_regex.match(line_indicator):
                            num_dialogues+=1

                        else:
                            print('fucking line ------- {}'.format(line))

                if num_dialogues == 0:
                    print("fucking dialogue: {}".format(dialogue))

                num_dialogues_convo.append(num_dialogues)
        return num_dialogues_convo

# cut out last 0 of the file
list_number_of_dialogues_train= count_dialogue_per_convo(train_source_file_directory)
print(list_number_of_dialogues_train)

list_number_of_dialogues_dev= count_dialogue_per_convo(dev_source_file_directory)
print(list_number_of_dialogues_dev)

list_number_of_dialogues_test= count_dialogue_per_convo(test_source_file_directory)
print(list_number_of_dialogues_test)

def combine_indices(lists_to_combine):
    # print(lists_to_combine[0])
    if not lists_to_combine:
        print("empty")
    required_length= len(lists_to_combine[0])
    # print('required length: {}'.format(required_length))
    for dialogue_list in lists_to_combine:
      assert(len(dialogue_list)==required_length)
    #create a new list without all 1s
    uncombined_list= [x for x in lists_to_combine if not statistics.mean(x)==1]

    if len(uncombined_list)==0:
        return [1 for i in range(required_length)]

    # initialize combined list as all 0
    combined_list = [0 for i in range(required_length)]
    for dialogue_list in lists_to_combine:
        for index,binary_value in enumerate(dialogue_list):
            if binary_value == 1:
                combined_list[index]=1
    return combined_list

def combine_indices_to_file(source_file, target_file, num_indices_list):
    with open(source_file,'r') as reader, open(target_file,'w') as writer:
        conversation_indices_lists=[]
        # source_text= reader.read(source_file)
        for index,num_dialogue in enumerate(num_indices_list):
            to_be_combined_indices=[]
            if(num_dialogue==0):
                print('num dialogue empty with index {}'.format(index))
            for i in range(num_dialogue):
                to_be_combined_indices.append(ast.literal_eval(reader.readline()))
            # print(to_be_combined_indices)
            combined_indices= combine_indices(to_be_combined_indices)
            conversation_indices_lists.append(combined_indices)
        
        for item in conversation_indices_lists:
            writer.write('%s\n' %item )

combine_indices_to_file(dev_source_indices_directory,dev_target_indices_directory,list_number_of_dialogues_dev)
combine_indices_to_file(train_source_indices_directory,train_target_indices_directory,list_number_of_dialogues_train)
combine_indices_to_file(test_source_indices_directory,test_target_indices_directory,list_number_of_dialogues_test)


            
                 
